# Matrix Factorization

Matrix Factorization はリコメンドエンジンに用いられる機械学習アルゴリズムです。

身近なリコメンドエンジンの例は、通販サイトの"おすすめ商品"欄です。ユーザーAの過去の購入履歴から、似たような購買傾向を持つ他のユーザーBを割り出すことができます。ユーザーBが過去に購入した商品はユーザーAも欲しがる可能性が高いため、おすすめ商品として表示することで購入を促す仕組みです。

今回は量子アニーリングを用いて Matrix Factorization を実行します。  

参考文献: O’Malley, Daniel, et al. "Nonnegative/binary matrix factorization with a d-wave quantum annealer." PloS one 13.12 (2018): e0206653.

データセットとして MovieLens dataset (https://grouplens.org/datasets/movielens/100k/) を用います。  
1682種類の映画に対する943ユーザーによる100,000回分の評価(星1~星5)のデータセットです。

まずデータセットを読み込みます。

In [1]:
import pandas as pd
import numpy as np

dataDir = './Data/'

#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(dataDir + 'ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(dataDir + 'ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv(dataDir + 'ml-100k/u.item', sep='|', names=i_cols,
encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_train = pd.read_csv(dataDir + 'ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv(dataDir + 'ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

データセットは訓練用とテスト用に分かれています。  
中身は以下の通りです。"user_id" の示すユーザーが "movie_id" の示す映画に与えた評価が "rating" として記録されています。

In [2]:
ratings_train

,user_id,movie_id,rating,unix_timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
90565,943,1047,2,875502146
90566,943,1074,4,888640250
90567,943,1188,3,888640250
90568,943,1228,3,888640275


これを行列に成型します。
行が "user_id"、列が "movei_id"、各要素がその行と列に対応する "rating" となります。

ここでは問題サイズの観点から "user_id"、"movei_id" ともに100までのデータのみを抽出し、100 x 100 行列とします。

In [3]:
user_num = 100 # Max.943
item_num = 100 # Max.1680

def df_to_matrix(df, user_num, item_num, row_name, col_name, values):
    df1 = df[df[row_name] <= user_num]
    df2 = df1[df1[col_name] <= item_num]
    for i in range(1, user_num+1):
        if len(df2.loc[(df2.loc[:,row_name]==i)]) == 0:
            df2 = df2.append(pd.DataFrame([[i,1]], columns=[row_name, col_name])).fillna(0)
    for i in range(1, item_num+1):
        if len(df2.loc[(df2.loc[:,col_name]==i)]) == 0:
            df2 = df2.append(pd.DataFrame([[1,i]], columns=[row_name, col_name])).fillna(0)
    
    return np.array(df2.pivot(index = row_name, columns =col_name, values = values).fillna(0))
   
R = df_to_matrix(ratings_train, user_num, item_num, 'user_id', 'movie_id', 'rating')
print(R.shape)

R_test = df_to_matrix(ratings_test, user_num, item_num, 'user_id', 'movie_id', 'rating')
print(R_test.shape)

(100, 100)
(100, 100)


In [4]:
R

array([[5., 3., 4., ..., 4., 3., 5.],
       [4., 0., 0., ..., 0., 0., 5.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 3., ..., 5., 0., 5.],
       [0., 0., 0., ..., 0., 0., 0.]])

このような行列を Matrixf Factorization では下図のように分解します。

$$
V \approx WH \\
$$

<img src="./img/212_img.png" width="50%">

"features" はユーザーと映画がそれぞれ持つ特徴量を表しています。  
特徴量とは例えばユーザーについて"アクション映画が好き"というような好みや傾向を表します。  映画の特徴量も同様に"アクション映画か否か"といった傾向を表すことができます。  
このように定式化すると、アクション映画が好きなユーザーはまだ見ぬアクション映画に対しても高い評価をつけると予想します。  

Matrix Factorization では $V \approx WH $ をなるべく精度良く満たせるような $W, H$ を求めます。  
特に $W$ を正の値とし、$H$ をバイナリ(0 または 1)値に限定した問題は Nonnegative/Binary Matrix Factorization (NBMF) と呼ばれます。  

NBMF は以下の最適化を繰り返し $W, H$ を更新していきます。

$$
W = \mathrm{argmin}_{X \in \mathbb{R}+^{M\times K}} \| V - XH \|_F  + \alpha \| X \|_F
$$

$$
H = \mathrm{argmin}_{X \in \{0,1\}^{K\times N} } \|V - WX \|_F
$$

さらに、$H$ の各列 $i$ ($H_i$) は次式のように独立に最適化できます。

$$
H_i = \mathrm{argmin}_{q \in \{0,1\}^k } \|V_i-Wq \|_F
$$

$\| \cdot \|_F$ はフロベニウスノルムを表します。  
$H$ はバイナリとしていたため、$q$ を求めるために量子アニーリングを活用できます。  
具体的には線形最小二乗問題に帰着でき、以下のようなQUBO定式化が知られています。  

$$
f(q) = \sum_i a_i q_i + \sum_{i<j} b_{ij}q_i q_j  \\
a_j = \sum_k W_{kj}(W_{kj} - 2V_{ki}) \\
b_{jk} = 2\sum_l W_{lj}W_{lk}
$$

参考: Borle, Ajinkya, and Samuel J. Lomonaco. "Analyzing the quantum annealing approach for solving linear least squares problems." International Workshop on Algorithms and Computation. Springer, Cham, 2019.

### 実装

blueqat で実装してみましょう。  
以下は量子アニーリングとQAOAの両方で実行できるようにしています。

In [20]:
import blueqat.wq as wq
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

class MF():

    def __init__(self, V, K, alpha, iterations, Aneal_iteration, method):
        self.V = V
        self.num_users, self.num_items = V.shape
        self.K = K
        self.alpha = alpha
        self.iterations = iterations
        self.iteAneal = Aneal_iteration
        self.step = 5 # QAOA step
        self.method = method

    def train(self):
        # Initializing user-feature and movie-feature matrix 
        self.b = np.mean(self.V[np.where(self.V != 0)]) # bias term
        self.W = np.abs((np.random.normal(scale=self.b/self.K, size=(self.num_users, self.K))))
        self.H = np.abs((np.random.normal(scale=1./2, size=(self.K, self.num_items))))

        self.predictedIndex = []
        self.itemVecs = []
        self.itemVecs_indices = []
        for j in range(self.num_items):
            self.itemVecs.append([])
            self.itemVecs_indices.append([])
            for i in range(self.num_users):
                if self.V[i, j] > 0:
                    self.itemVecs[j].append(self.V[i, j]) # jth column's non 0 elements 
                    self.itemVecs_indices[j].append(i) # jth column's non 0 indices
        
        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            #np.random.shuffle(self.samples)
            self.train_W()
            if self.method == 'QAOA':
                self.train_H_QAOA()
            elif self.method == 'Anneal':
                self.train_H_Anneal()
            aveError = self.aveError()
            if (i+1) % 1 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, aveError))
            if i>0 and np.abs(aveError - training_process[-1][1]) < aveError/50: # convergence
                break
            else:
                training_process.append((i, aveError))
        return training_process

    # Computing mean error
    def aveError(self):
        xs, ys = self.V.nonzero()
        predicted = self.full_matrix()
        error = 0
        l = len(xs)
        for x, y in zip(xs, ys):
            error += np.abs(self.V[x, y] - predicted[x, y])
        return error / l

    # Stochastic gradient descent to get optimized W matrix
    def train_W(self):
        self.Wdiff = np.zeros([self.num_users, self.K])
        for i in range(self.num_users):
            for q in range(self.K):
                for j in range(self.num_items):
                    if self.V[i][j] == 0:
                        continue
                    self.Wdiff[i][q] += 2 * (self.V[i][j] - np.dot(self.W[i,:], self.H[:,j])) * (-1 * self.H[q][j])
        self.W = np.abs(self.W - self.alpha * self.Wdiff)

    def train_H_QAOA(self):
        # use QAOA method
        for j in range(self.num_items): #jth column of V    
            # define QUBO hamiltonian and solve
            self.hamiltonian = 0
            for l in range(self.K):
                for m in range(l,self.K):
                    if l==m: # diagonal elements
                        for k in self.itemVecs_indices[j]: #jth columns's k's element
                            self.hamiltonian += np.float(self.W[k][l] * (self.W[k][l] - 2 * self.V[k][j])) * q(l)
                    else:
                        for k in self.itemVecs_indices[j]:
                            self.hamiltonian += 2 * np.float(self.W[k][l] * self.W[k][m]) * q(l) * q(m)
            result = vqe.Vqe(vqe.QaoaAnsatz(self.hamiltonian, self.step)).run()
            self.H[:,j] = list(result.most_common(1)[0][0])
            
    def train_H_Anneal(self):
        # use Q-Anealing
        self.JthAneal = 0
        for j in range(self.num_items): #jth column of V
            for ite in range(self.iteAneal): # iteration of anealing
                # define QUBO and solve
                self.a = wq.Opt()
                self.a.qubo = l_2d_ok = [[0] * self.K for i in range(self.K)] # initialize qubo matrix
                for l in range(self.K):
                    for m in range(l,self.K):
                        if l==m: # diagonal elements
                            for k in self.itemVecs_indices[j]: #jth columns's k's element
                                self.a.qubo[l][m] += self.W[k][l] * (self.W[k][l] - 2 * self.V[k][j])
                        else:
                            for k in self.itemVecs_indices[j]:
                                self.a.qubo[l][m] += 2 * self.W[k][l] * self.W[k][m]
                
                tmp = self.a.sa()
                tmp_error = self.error_aneal(tmp)
                if ite == 0:
                    self.anealBit = tmp
                    min_error = self.error_aneal(tmp)
                else: # update or not
                    if tmp_error < min_error:
                        self.anealBit = tmp
            
            self.H[:,j] = self.anealBit
                
    def error_aneal(self, anealBit):
        F = 0
        for l in range(self.K):
            for m in range(l,self.K):
                if l==m: # diagonal elements
                    F += self.a.qubo[l][m] * anealBit[l]
                else:
                    F += self.a.qubo[l][m] * anealBit[l] * anealBit[m]
        return abs(F)        

    # Full user-movie rating matrix
    def full_matrix(self):
        return self.W.dot(self.H)

### 学習

学習を行います。  
エラーの減少が一定程度収束したら学習を終了します。

In [21]:
np.random.seed = 42
mf = MF(R, K=5, alpha=0.01, iterations=20, Aneal_iteration=1, method='Anneal')
training_process = mf.train()
print()
print("W x H:")
print(mf.full_matrix())

Iteration: 1 ; error = 1.1610
Iteration: 2 ; error = 0.8512
Iteration: 3 ; error = 0.8231
Iteration: 4 ; error = 0.7669
Iteration: 5 ; error = 0.8519
Iteration: 6 ; error = 0.7382
Iteration: 7 ; error = 0.7189
Iteration: 8 ; error = 0.7329

W x H:
[[3.07752757 1.57568632 2.76098952 ... 3.07752757 2.21578174 3.29777999]
 [3.12600217 3.17252369 2.18554561 ... 3.12600217 2.10711165 3.93359492]
 [1.68016056 2.22742145 0.92386796 ... 1.68016056 2.23645789 2.994783  ]
 ...
 [3.03236212 3.19494034 1.54332623 ... 3.03236212 3.34842419 3.8458737 ]
 [4.35070465 3.87558426 2.96300904 ... 4.35070465 3.45499378 5.10746399]
 [1.17751116 1.72820597 0.50113742 ... 1.17751116 1.69326523 2.03189509]]


### 予測性能の確認

学習したユーザーと映画の特徴量を用いて、ユーザーのまだ見ていない映画に対する評価を予測しましょう。  
予測するユーザーと映画の組み合わせはテストデータに含まれているものとし、テストデータにおける評価を正解とします。

In [22]:
prediction = mf.full_matrix()

In [23]:
predictedRating = []
testRating = []
score = [[],[],[],[],[]]

xs, ys = R_test.nonzero()

error_test = 0
for x,y in zip(xs, ys):
    predictedRating.append(prediction[x][y])
    testRating.append(R_test[x][y])
    score[int(R_test[x][y] - 1)].append(min(prediction[x][y],5))

    error_test += np.abs(R_test[x][y] - prediction[x][y])
error_test = error_test / len(xs)

テストデータで評価が 1, 2, 3, 4, 5点だったデータの平均予測点をそれぞれ算出してみましょう。  

In [31]:
print('Test error =', error_test)
print()
print('Test dataでそれぞれ1,2,3,4,5点だった箇所の予測点数平均')
print(np.mean(score[0]), '\n',
     np.mean(score[1]), '\n',
     np.mean(score[2]), '\n',
     np.mean(score[3]), '\n',
     np.mean(score[4]),)   

Test error = 0.8367807157320516

Test dataでそれぞれ1,2,3,4,5点だった箇所の予測点数平均
2.473954521232848 
 3.3221181082768103 
 3.5287032057191428 
 3.6094973953846186 
 4.093452207661236


全体としては、テストデータと予測結果で高評価/低評価の傾向は合っていることがわかります。  
今回の条件における学習では 2, 3, 4点の判別は難しそうですが、１点と5点はそれなりに判別できそうです。  

以上より、量子アニーリング/QAOAを用いて Matrix Factorization を用いた推薦アルゴリズムを実装しました。

In [34]:
error_test

0.8367807157320516